<a href="https://colab.research.google.com/github/semih108/CarPricePredictor/blob/main/ScrapingCarData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_page_data(soup, data_list, make, model):
    for car in soup.find_all('div', class_='ListItem_wrapper__TxHWu'):
        try:
            name = car.find('h2').get_text(strip=True)
        except AttributeError:
            name = "N/A"

        try:
            price = car.find('p', class_='Price_price__APlgs').get_text(strip=True)
        except AttributeError:
            price = "N/A"

        try:
            details = car.find('div', class_='VehicleDetailTable_container__XhfV1')
            mileage = details.find('span', {'data-testid': 'VehicleDetails-mileage_road'}).get_text(strip=True)
            transmission = details.find('span', {'data-testid': 'VehicleDetails-transmission'}).get_text(strip=True)
            date = details.find('span', {'data-testid': 'VehicleDetails-calendar'}).get_text(strip=True)
            fuel = details.find('span', {'data-testid': 'VehicleDetails-gas_pump'}).get_text(strip=True)
            power = details.find('span', {'data-testid': 'VehicleDetails-speedometer'}).get_text(strip=True)
        except AttributeError:
            mileage = transmission = date = fuel = power = "N/A"

        data_list.append([make, model, name, price, mileage, transmission, date, fuel, power])

def scrape_autoscout24(make, model):
    data_list = []
    params = {
        'atype': 'C',
        'page': 1,
        'make': make,
        'model': model,
        'cy': 'D', # change country here: A, D
        'damaged_listing': 'exclude',
        'desc': '0',
        'powertype': 'kw',
        'sort': 'standard',
        'ustate': 'N,U'
    }
    base_url = f'https://www.autoscout24.at/lst/{make}'
    while 1:
        response = requests.get(f"{base_url}/{model}", params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        get_page_data(soup, data_list, make, model)
        next_button = soup.find('button', {'aria-label': 'Zu nächsten Seite'})
        if next_button and 'disabled' not in next_button.attrs.get('class', []):
            print(f"Next button found on page {params['page']}")
            params['page'] += 1
            time.sleep(2)
        else:
            print("No next button found or it is disabled.")
            break

    return data_list

In [ ]:
car_models = {
    'audi': [
        'a5', '100', '200', '50', '80', '90', 'a1', 'a2', 'a3', 'a4', 'a4-allroad', 'a5', 'a6', 'a6-allroad', 'a7', 'a8',
        'allroad', 'cabriolet', 'coupe', 'e-tron', 'e-tron-gt', 'q1', 'q2', 'q3', 'q4-e-tron', 'q5', 'q6', 'q7', 'q8',
        'q8-e-tron', 'quattro', 'r8', 'rs', 'rs-e-tron-gt', 'rs-q3', 'rs-q5', 'rs-q8', 'rs2', 'rs3', 'rs4', 'rs5', 'rs6',
        'rs7', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 'sonstige', 'sq2', 'sq3', 'sq5', 'sq6', 'sq7', 'sq8',
        'sq8-e-tron', 'tt', 'tt-rs', 'tts', 'v8'
    ],
    'bmw': [
        "114", "116", "118", "120", "123", "125", "128", "130", "135", "140",
        "2002", "214", "216", "218", "220", "223", "225", "228", "230", "235", "240",
        "315", "316", "318", "320", "323", "324", "325", "328", "330", "335", "340", "Active Hybrid 3",
        "418", "420", "425", "428", "430", "435", "440",
        "518", "520", "523", "524", "525", "528", "530", "535", "540", "545", "550", "Active Hybrid 5",
        "620", "628", "630", "633", "635", "640", "645", "650",
        "725", "728", "730", "732", "735", "740", "745", "750", "760", "Active Hybrid 7",
        "830", "840", "850", "i3", "i4", "i5", "i7", "i8", "iX", "iX1", "iX2", "iX3",
        "1er M Coupé", "M1", "M2", "M3", "M4", "M5", "M550", "M6", "M8", "M850",
        "Active Hybrid X6", "X1", "X2", "X2 M", "X3", "X3 M", "X4", "X4 M", "X5", "X5 M", "X6", "X6 M", "X7", "X7 M", "XM",
        "Z1", "Z3", "Z3 M", "Z4", "Z4 M", "Z8", "Sonstige"
    ],
    'mercedes-benz': [
        "A 180", "A 200", "A 220", "A 250", "A 35 AMG", "A 45 AMG",
        "B 180", "B 200", "B 220", "B 250", "B Electric Drive",
        "C 180", "C 200", "C 220", "C 250", "C 300", "C 350", "C 43 AMG", "C 63 AMG",
        "CLA 180", "CLA 200", "CLA 220", "CLA 250", "CLA 35 AMG", "CLA 45 AMG",
        "CLS 350", "CLS 450", "CLS 53 AMG", "CLS 63 AMG",
        "E 200", "E 220", "E 250", "E 300", "E 350", "E 400", "E 450", "E 53 AMG", "E 63 AMG",
        "G 350", "G 400", "G 500", "G 63 AMG", "G 65 AMG",
        "GLA 180", "GLA 200", "GLA 220", "GLA 250", "GLA 35 AMG", "GLA 45 AMG",
        "GLB 200", "GLB 220", "GLB 250", "GLB 35 AMG",
        "GLC 200", "GLC 220", "GLC 250", "GLC 300", "GLC 43 AMG", "GLC 63 AMG",
        "GLE 250", "GLE 300", "GLE 350", "GLE 400", "GLE 450", "GLE 500", "GLE 53 AMG", "GLE 63 AMG",
        "GLS 350", "GLS 400", "GLS 450", "GLS 500", "GLS 580", "GLS 63 AMG", "Maybach GLS",
        "S 350", "S 400", "S 450", "S 500", "S 560", "S 580", "S 600", "S 63 AMG", "S 65 AMG", "Maybach S-Klasse",
        "SL 400", "SL 450", "SL 500", "SL 550", "SL 63 AMG", "SL 65 AMG",
        "SLC 180", "SLC 200", "SLC 300", "SLC 43 AMG",
        "V 200", "V 220", "V 250",
        "EQA", "EQB", "EQC", "EQE", "EQS", "EQV",
        "CLA 35 AMG", "CLA 45 AMG", "CLC", "CLE", "CLE 200", "CLE 220", "CLE 300", "CLE 450", "CLE 530 AMG",
        "CLK 200", "CLK 220", "CLK 230", "CLK 240", "CLK 270", "CLK 280", "CLK 320", "CLK 350", "CLK 430", "CLK 500", "CLK 550 AMG", "CLK 630 AMG",
        "CLS 220", "CLS 250", "CLS 280", "CLS 300", "CLS 320", "CLS 400", "CLS 450", "CLS 500", "CLS 530 AMG", "CLS 550 AMG", "CLS 630 AMG",
        "E 230", "E 240", "E 260", "E 270", "E 280", "E 290", "E 320", "E 360 AMG", "E 420", "E 430 AMG", "E 430", "E 500 AMG", "E 530 AMG", "E 550 AMG", "E 550", "E 600 AMG", "E 630 AMG",
        "EQ-A 250", "EQ-A 300", "EQ-A 350", "EQ-B 250", "EQ-B 300", "EQ-B 350", "EQ-C 400", "EQ-E 300", "EQ-E 350", "EQ-E 430", "EQ-E 500", "EQ-E 530", "EQ-E SUV", "EQ-S SUV",
        "EQ-T", "EQ-V 250", "EQ-V 300", "G 230", "G 240", "G 250", "G 270", "G 280", "G 290", "G 320", "G 450", "G 550 AMG", "G 580", "G 630 AMG", "G 650 AMG",
        "GL 320", "GL 350", "GL 400", "GL 420", "GL 450", "GL 500", "GL 550 AMG", "GL 630 AMG",
        "GLA 350 AMG", "GLA 450 AMG", "GLB 180", "GLB 350 AMG", "GLC 350", "GLC 400", "GLC 430 AMG", "GLC 450", "GLC 630 AMG",
        "GLE 430 AMG", "GLE 530 AMG", "GLE 580", "GLE 630 AMG", "GLK 200", "GLK 220", "GLK 250", "GLK 280", "GLK 300", "GLK 320", "GLK 350",
        "GLS 580", "GLS 600", "GLS 630 AMG", "ML 230", "ML 250", "ML 270", "ML 280", "ML 300", "ML 320", "ML 350", "ML 400", "ML 420", "ML 430", "ML 450", "ML 500", "ML 550 AMG", "ML 630 AMG",
        "Marco Polo", "MB 100", "R 280", "R 300", "R 320", "R 350", "R 500", "R 630 AMG", "S 250", "S 260", "S 280", "S 300", "S 320"
    ],
    'volkswagen': [
        "181", "Amarok", "Anfibio", "Arteon", "Atlas", "Beetle", "Bora", "Buggy", "Bus", "Caddy", "CC", "Coccinelle",
        "Corrado", "Crafter", "Cross Touran", "Derby", "e-up!", "Eos", "Escarabajo", "Fox", "Cross Golf", "e-Golf",
        "Golf", "Golf Cabriolet", "Golf GTD", "Golf GTE", "Golf GTI", "Golf Plus", "Golf R", "Golf Sportsvan", "Golf Variant",
        "Grand California", "ID. Buzz", "ID. Buzz Cargo", "ID.3", "ID.4", "ID.5", "ID.6", "ID.7", "Iltis", "Jetta",
        "Käfer", "Karmann Ghia", "Kever", "L80", "LT", "Lupo", "Maggiolino", "New Beetle", "Passat", "Passat Alltrack",
        "Passat CC", "Passat Variant", "Phaeton", "Pointer", "Polo", "Polo Cross", "Polo GTI", "Polo Plus", "Polo R WRC",
        "Polo Sedan", "Polo Variant", "Routan", "Santana", "Scirocco", "Sharan", "T-Cross", "T-Roc", "T1", "T2",
        "T3", "T3 Blue Star", "T3 California", "T3 Caravelle", "T3 Kombi", "T3 Multivan", "T3 White Star", "T4",
        "T4 Allstar", "T4 California", "T4 Caravelle", "T4 Kombi", "T4 Multivan", "T5", "T5 California", "T5 Caravelle",
        "T5 Kombi", "T5 Multivan", "T5 Shuttle", "T5 Transporter", "T6 California", "T6 Caravelle", "T6 Kombi",
        "T6 Multivan", "T6 Transporter", "T6.1 California", "T6.1 Caravelle", "T6.1 Kombi", "T6.1 Multivan",
        "T6.1 Transporter", "T7 California", "T7 Kastenwagen", "T7 Multivan", "Taigo", "Taro", "Tiguan", "Tiguan Allspace",
        "Touareg", "Touran", "Transporter", "up!", "Vento", "Viloran", "XL1", "Sonstige"
    ],
    'skoda': [
        "105", "120", "130", "135", "Citigo", "Enyaq", "Fabia", "Favorit", "Felicia", "Forman", "Kamiq", "Karoq", "Kodiaq",
        "Octavia", "Pick-up", "Praktik", "Rapid/Spaceback", "Roomster", "Scala", "Snowman", "Superb", "Yeti", "Sonstige"
    ],
    'ford': [
        'Aerostar', 'B-Max', 'Bronco', 'C-Max', 'Capri', 'Connect Elektro', 'Consul', 'Cougar', 'Courier', 'Crown',
        'Customline', 'Econoline', 'Econovan', 'EcoSport', 'Edge', 'Escape', 'Escort', 'Excursion', 'Expedition',
        'Explorer', 'Express', 'F 1', 'F 100', 'F 150', 'F 250', 'F 350', 'F 360', 'F 450', 'F 550', 'F 650',
        'F Super Duty', 'Fairlane', 'Falcon', 'Fiesta', 'Flex', 'Focus', 'Focus C-Max', 'Focus CC', 'Freestar',
        'Freestyle', 'Fusion', 'Galaxy', 'Gran Torino', 'Granada', 'Grand C-Max', 'Grand Tourneo', 'GT', 'Ka/Ka+',
        'Kuga', 'M', 'Maverick', 'Mercury', 'Mondeo', 'Mustang', 'Mustang Mach-E', 'Orion', 'Probe', 'Puma',
        'Ranger', 'Ranger Raptor', 'RS 200', 'S-Max', 'Scorpio', 'Sierra', 'Sportka', 'Streetka', 'Taunus',
        'Taurus', 'Thunderbird', 'Torino', 'Tourneo', 'Tourneo Connect', 'Tourneo Courier', 'Tourneo Custom',
        'Transit', 'E-Transit', 'Transit Bus', 'Transit Connect', 'Transit Courier', 'Transit Custom', 'Windstar',
        'Sonstige'
    ],
    'toyota': [
        '4-Runner', 'Allion', 'Alphard', 'Altezza', 'Aristo', 'Auris', 'Avalon', 'Avensis', 'Avensis Verso', 'Aygo',
        'Aygo X', 'BB', 'Belta', 'bZ4X', 'C-HR', 'Caldina', 'Cami', 'Camry', 'Carina', 'Celica', 'Chaser', 'Coaster',
        'Corolla', 'Corolla Cross', 'Corolla Verso', 'Corona', 'Corsa', 'Cressida', 'Cresta', 'Crown', 'Duet', 'Dyna',
        'Estima', 'FJ Cruiser', 'FJ40', 'Fortuner', 'Fun Cruiser', 'Funcargo', 'Gaia', 'GR86', 'GT86', 'Harrier', 'HDJ',
        'Hiace', 'Highlander', 'Hilux', 'Ipsum', 'iQ', 'Ist', 'KJ', 'Land Cruiser', 'Land Cruiser Prado', 'Lite-Ace',
        'Mark II', 'Mark X', 'Matrix', 'Mirai', 'Model F', 'MR 2', 'Nadia', 'Noah', 'Opa', 'Paseo', 'Passo', 'Pick up',
        'Picnic', 'Platz', 'Premio', 'Previa', 'Prius', 'Prius+', 'Proace', 'Proace City', 'Ractis', 'Raum', 'RAV 4',
        'Sequoia', 'Sienna', 'Solara', 'Sprinter', 'Starlet', 'Supra', 'Tacoma', 'Tercel', 'Town Ace', 'Tundra',
        'Urban Cruiser', 'Venza', 'Verossa', 'Verso', 'Verso-S', 'Vista', 'Vitz', 'Voxy', 'Will', 'Windom', 'Wish',
        'Yaris', 'Yaris Cross', 'Sonstige'
    ],
    'citroen': [
        '2CV', 'Acadiane', 'Ami', 'AX', 'Axel', 'Berlingo', 'BX', 'C-Crosser', 'C-Elysée', 'C-Zero', 'C1', 'C15', 'C2',
        'C25', 'C3', 'C3 Aircross', 'C3 Picasso', 'C35', 'C4', 'C4 Aircross', 'C4 Cactus', 'C4 Picasso', 'C4 SpaceTourer',
        'C4 X', 'E-C4 Electric', 'E-C4 X', 'Grand C4 Picasso', 'Grand C4 SpaceTourer', 'C5 X', 'C5', 'C5 Aircross', 'C6',
        'C8', 'CX', 'DS', 'DS3', 'DS4', 'DS5', 'Dyane', 'E-Méhari', 'Evasion', 'GSA', 'Holidays', 'Jumper', 'Jumpy', 'LNA',
        'Méhari', 'Nemo', 'SAXO', 'SM', 'Spacetourer', 'Traction', 'Visa', 'Xantia', 'XM', 'Xsara', 'Xsara Picasso', 'ZX',
        'Sonstige'
    ],
    'porsche': [
    '356', '550',
    '718', '718 Spyder',
    '911', '930', '964', '991', '992', '993', '996', '997',
    '912', '914', '918', '924', '928', '944', '959', '962', '968',
    'Boxster', 'Carrera GT',
    'Cayenne', 'Cayman',
    'Macan', 'Panamera',
    'Targa', 'Taycan',
    'Sonstige'
    ],
    'mini': [
    '1000', '1300',
    '3/5-Türer', 'Cooper', 'Cooper D', 'Cooper S', 'Cooper SD', 'Cooper SE', 'John Cooper Works', 'One', 'One D',
    'Aceman',
    'Cabrio-Serie', 'Cooper Cabrio', 'Cooper D Cabrio', 'Cooper S Cabrio', 'Cooper SD Cabrio', 'John Cooper Works Cabrio', 'One Cabrio',
    'Clubman-Serie', 'Cooper Clubman', 'Cooper D Clubman', 'Cooper S Clubman', 'Cooper SD Clubman', 'John Cooper Works Clubman', 'One Clubman', 'One D Clubman', 'Clubvan',
    'Countryman-Serie', 'Cooper Countryman', 'Cooper D Countryman', 'Cooper S Countryman', 'Cooper SD Countryman', 'Cooper SE Countryman', 'Countryman C', 'Countryman D', 'Countryman S All4', 'JCW Countryman All4', 'John Cooper Works Countryman', 'One Countryman', 'One D Countryman',
    'Coupe-Serie', 'Cooper Coupe', 'Cooper D Coupe', 'Cooper S Coupe', 'Cooper SD Coupe', 'John Cooper Works Coupe',
    'Paceman-Serie', 'Cooper Paceman', 'Cooper D Paceman', 'Cooper S Paceman', 'Cooper SD Paceman', 'John Cooper Works Paceman',
    'Roadster-Serie', 'Cooper Roadster', 'Cooper S Roadster', 'Cooper SD Roadster', 'John Cooper Works Roadster',
    'Sonstige'
    ],
    'abarth': [
    '124 GT', '124 Rally Tribute', '124 Spider',
    '500', '500C', '500e',
    '595', '595 Competizione', '595 Pista', '595 Turismo', '595C',
    '695', '695C',
    'Grande Punto', 'Punto EVO', 'Punto Supersport',
    'Sonstige'
    ],
    'alfa-romeo': [
    '145', '146', '147', '155', '156', '159',
    '164', '166', '1750', '2000', '33', '4C',
    '75', '8C', '90', 'Alfa 6', 'Alfasud', 'Alfetta',
    'Brera', 'Crosswagon', 'Giulia', 'Giulietta', 'GT', 'GTV',
    'Junior', 'MiTo', 'Montreal', 'Quadrifoglio', 'RZ', 'Spider',
    'Sportwagon', 'Sprint', 'Stelvio', 'SZ', 'Tonale', 'Sonstige'
    ],
    'aston-martin': [
        'AR1', 'Cygnet', 'DB', 'DB11', 'DB12', 'DB7', 'DB9', 'DBS', 'DBX',
        'Lagonda', 'Rapide', 'V8', 'Valkyrie', 'Vanquish', 'Vantage',
        'Virage', 'Volante', 'Sonstige'
    ],
    'seat': [
        'Arona', 'Ateca', 'Born', 'Formentor', 'Formentor VZ5',
        'Ibiza', 'Leon', 'Tavascan', 'Sonstige'
    ],
    'fiat': [
        '124 Coupè', '124 Spider', '126', '127', '128', '130', '131', '132', '133', '2300', '242',
        '500', '500 Abarth', '500C', '500C Abarth', '500e', '500L', '500X', '595 Abarth',
        '600', '850', '900', 'Albea', 'Argenta', 'Barchetta', 'Brava', 'Bravo',
        'Campagnola', 'Cinquecento', 'Coupe', 'Croma', 'Dino', 'Doblo', 'E-Doblo',
        'Ducato', 'Duna', 'E-Ulysse', 'Fiorino', 'Freemont', 'Fullback', 'Grande Punto',
        'Idea', 'Linea', 'Marea', 'Marengo', 'Maxi', 'Multipla', 'New Panda',
        'Palio', 'Panda', 'Penny', 'Pininfarina', 'Punto', 'Punto Evo', 'Qubo',
        'Regata', 'Ritmo', 'Scudo', 'Sedici', 'Seicento', 'Spider Europa',
        'Stilo', 'Strada', 'Talento', 'Tempra', 'Tipo', 'Topolino', 'Ulysse', 'Uno', 'X 1/9', 'Sonstige'
    ],
    'jaguar': [
        '420', 'D-Type', 'Daimler', 'E-Pace', 'E-Type',
        'F-Pace', 'F-Type', 'I-Pace', 'MK II', 'S-Type',
        'Sovereign', 'X-Type', 'X300', 'XE', 'XF',
        'XJ', 'XJ12', 'XJ40', 'XJ6', 'XJ8', 'XJR',
        'XJS', 'XJSC', 'XK', 'XK8', 'XKR',
        'Sonstige'
    ],
    'jeep': [
        'Avenger', 'Cherokee', 'CJ-5', 'CJ-7', 'CJ-8',
        'Comanche', 'Commander', 'Compass', 'Gladiator', 'Grand Cherokee',
        'Liberty', 'Patriot', 'Renegade', 'Wagoneer', 'Willys',
        'Wrangler', 'Sonstige'
    ],
    'kia': [
        'Besta', 'Carens', 'Carnival', 'Ceed / cee\'d', 'Ceed SW / cee\'d SW',
        'Cerato', 'Clarus', 'e-Niro', 'Elan', 'EV6', 'EV9',
        'Joice', 'K2500', 'K2700', 'K2900', 'Leo', 'Magentis', 'Mentor',
        'Mohave/Borrego', 'Niro', 'Opirus', 'Optima', 'Picanto', 'Pregio',
        'Pride', 'ProCeed / pro_cee\'d', 'Retona', 'Rio', 'Roadster', 'Rocsta',
        'Sephia', 'Shuma', 'Sorento', 'Soul', 'Spectra', 'Sportage', 'Stinger',
        'Stonic', 'Venga', 'XCeed', 'Sonstige'
    ],
    'nissan': [
        '100 NX', '200 SX', '280 ZX', '300 ZX', '350Z', '370Z',
        'AD', 'Almera', 'Almera Tino', 'Altima', 'Ariya', 'Armada', 'Avenir', 'Bassara',
        'Bluebird', 'Cabstar', 'Cargo', 'Cedric', 'Cefiro', 'Cherry', 'Cube', 'Datsun',
        'E-NV200', 'Elgrand', 'Evalia', 'Expert', 'Figaro', 'Frontier', 'Gloria',
        'GT-R', 'Interstar', 'Juke', 'King Cab', 'Kubistar', 'Laurel', 'Leaf',
        'Liberty', 'March', 'Maxima', 'Micra', 'Murano', 'Navara', 'Note', 'NP300',
        'NV200', 'NV250', 'NV300', 'NV400', 'Pathfinder', 'Patrol', 'Pick Up',
        'Pixo', 'Prairie', 'Presage', 'Presea', 'Primastar', 'Primera', 'Pulsar',
        'Qashqai', 'Qashqai+2', 'Quest', 'R Nessa', 'Rogue', 'Safari', 'Sentra',
        'Serena', 'Silvia', 'Skyline', 'Stagea', 'Stanza', 'Sunny', 'Teana',
        'Terrano', 'Tiida', 'Titan', 'Townstar', 'Townstar EV', 'Trade', 'Urvan',
        'Vanette', 'Wingroad', 'X-Trail', 'Sonstige'
    ],
    'peugeot': [
        '1007', '104', '106', '107', '108', '2008', '204', '205', '206', '207',
        '208', '3008', '301', '304', '305', '306', '307', '308', '309',
        '4007', '4008', '404', '405', '406', '407', '408', '5008', '504',
        '505', '508', '604', '605', '607', '806', '807', 'Bipper', 'Boxer',
        'Camper', 'e-2008', 'e-208', 'e-Expert', 'e-Rifter', 'Expert', 'iOn',
        'J5', 'J9', 'Partner', 'Ranch', 'RCZ', 'Rifter', 'Traveller', 'Sonstige'
    ],
    'renault': [
        'Alaskan', 'Alpine A110', 'Alpine A310', 'Alpine A610', 'Alpine V6', 'Arkana', 'Austral', 'Avantime',
        'Captur', 'Clio', 'Coupe', 'Duster', 'Espace', 'Express', 'Fluence', 'Fluence Z.E.', 'Fuego',
        'Grand Espace', 'Grand Modus', 'Grand Scenic', 'Kadjar', 'Kangoo', 'Kangoo E-TECH', 'Kangoo Z.E.',
        'Koleos', 'Laguna', 'Latitude', 'Logan', 'Mascott', 'Master', 'Megane', 'Megane E-Tech', 'Messenger',
        'Modus', 'P 1400', 'R 11', 'R 14', 'R 18', 'R 19', 'R 20', 'R 21', 'R 25', 'R 30', 'R 4', 'R 5',
        'R 6', 'R 9', 'Rafale', 'Rapid', 'Safrane', 'Sandero', 'Sandero Stepway', 'Scenic', 'Spider',
        'Super 5', 'Symbioz', 'Symbol', 'Talisman', 'Trafic', 'Twingo', 'Twizy', 'Vel Satis', 'Wind', 'ZOE',
        'Sonstige'
    ],
    'seat': [
        'Alhambra', 'Altea', 'Altea XL', 'Arona', 'Arosa', 'Ateca', 'Cordoba',
        'Exeo', 'Fura', 'Ibiza', 'Inca', 'Leon', 'Leon e-Hybrid',
        'Malaga', 'Marbella', 'Mii', 'Panda', 'Ronda', 'Tarraco',
        'Terra', 'Toledo', 'Sonstige'
    ],
    'smart': [
        '#1', '#3', 'brabus', 'city-coupé/city-cabrio', 'crossblade',
        'forFour', 'forTwo', 'roadster', 'Sonstige'
    ],
    'suzuki': [
        'Across', 'Alto', 'Baleno', 'Cappuccino', 'Carry', 'Celerio',
        'Escudo', 'Grand Vitara', 'Ignis', 'Ik-2', 'Jimny', 'Kizashi', 'Liana',
        'LJ 80', 'Maruti', 'S-Cross', 'SA 310', 'Samurai', 'Santana',
        'SJ 410', 'SJ 413', 'SJ Samurai', 'Splash', 'Super-Carry', 'Swace',
        'Swift', 'SX4', 'SX4 S-Cross', 'Vitara', 'Wagon R+', 'X-90', 'XL-7', 'Sonstige'
    ],
    'volvo': [
        '240', '244', '245', '262', '264', '265', '340', '360', '440', '460', '480',
        '740', '744', '745', '760', '764', '780', '850', '855', '940', '944', '945',
        '960', '965', 'Amazon', 'C30', 'C40', 'C70', 'EC40', 'EX30', 'EX40', 'EX90',
        'P1800', 'Polar', 'PV544', 'S40', 'S60', 'S60 Cross Country', 'S70', 'S80',
        'S90', 'V40', 'V40 Cross Country', 'V50', 'V60', 'V60 Cross Country', 'V70',
        'V90', 'V90 Cross Country', 'XC40', 'XC60', 'XC70', 'XC90', 'Sonstige'
    ]

}

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

# deleted 'bmw', 'audi', because we already have it
makeNames = [
     'mercedes-benz', 'volkswagen', 'skoda',
    'ford', 'toyota', 'citroen', 'porsche', 'mini', 'abarth',
    'alfa-romeo', 'aston-martin', 'seat', 'jeep', 'kia',
    'nissan', 'peugeot', 'renault', 'smart', 'suzuki', 'volvo'
]


for makeName in makeNames:
  all_data = [["Make", "Model", "Name", "Price", "Mileage", "Transmission", "Date", "Fuel", "Power"]]

  models = car_models.get(makeName, [])
  for model in models:
      print(f"Scraping {makeName} {model}")
      data = scrape_autoscout24(makeName, model)
      all_data.extend(data)

  # Save the data to a CSV file
  with open(f'/content/drive/MyDrive/scraped_car_data_at_111224/{makeName}_car_prices_at_111224.csv', 'w', newline='', encoding='utf-8') as file:
      writer = csv.writer(file)
      writer.writerows(all_data)